# Funkcje i elementy języka funkcyjnego

W laboratorium zostaną omówione najważniejsze elementy związane z funkcjami i elementami pochodnymi jak funktory i generatory. Jako materiał uzupełniający warto wrócić do laboratorium 1, gdzie zostały przedstawione dwie dodatkowe metody przekazywania parametrów, `*` i `**`.

## 1. Przekazywanie parametrów

Funkcja to zestaw instrukcji z wydzielonymi parametrami wejściowymi o zasięgu lokalnym, identyfikatorem (nazwą funkcji) oraz zwracanym wyniku, który może przyjmować wartość `None`. Wyrażenia `lambda` łamią tą zasadę, gdyż nie posiadają nazwy, a jedynie są przypisywane do zmiennej, która przechowuje miejsce w pamięci, gdzie funkcja jest przechowywana. Funkcje mogą być globalne (w ramach swojego modułu), importowane z poziomu innych plików języka Python bezpośrednio po nazwie, agregowane w klasach (jako podstawowy składnik klasy) lub zagnieżdżone (_nested_). Dwa typy zostaną omówione w kolejnych sekcjach.

Poniżej znajdują się trzy proste funkcje, (i) nie zwracająca żadnego wyniku, (ii) zwracająca wynik w postaci zmiennej typu `int` oraz (iii) zwracająca `tuple`.

In [ ]:
def func_1():
    pass # lub return

def func_2(a: int, b: int):
    return a + b

def func_3(a: int, b: int):
    return a+b, a*b # skrócony zapis (a+b, a*b)

Podstawowe anotacje zostały już omówione w laboratorium 1. Ich stosowanie jest zalecane z dwóch powodów: w ten sposób kod sam się dokumentuje (wiadomo jakiego typu mają być przekazywane parametry) oraz dla bezpieczeństwa (istnieje wiele programów sprawdzających poprawność kodu np. `mypy`). Fizycznie anotacje są jedynie wpisami w słowniku `__annotatations__` i z punktu widzenia kompilacji i wykonywania kodu bez znaczenia. W kolejnej sekcji zostanie przedstawiony przykład jak anotacje są przypisywane do funkcji oraz jak można wykorzystać tę informację w dekoratorach funkcji.

Oprócz prostych anotacji, w języku Python można deklarować również typu bardziej złożone np. słowniki czy listy, których elementy również mogą być typami złożonymi np. tuplami. Poniżej przykład paru typów bardziej złożonych.

In [ ]:
from typing import List, Dict, Tuple, Any

def func1(par1: Dict[str, List[Tuple[str, Any]]]):
    pass


W powyższym przykładzie charakterystycznymi elementami są nawiasy kwadratowe, w których zagnieżdżonych typów używa się jako parametrów. Zmienna `par1` powinna być typu słownikowego, którego klucz jest łańcuchem znaków, a wartością jest lista tupli, której pierwszym elementem jest łańcuch znaków, a drugim dowolny, nieznany typ.

W tym miejscu warto również wspomnieć o dwóch anotacjach: `union` i `optional`. Jak zostało wspomniane w laboratorium 1, w języku Python nie ma silnego typowania i w związku z tym nie ma przeciążenia funkcji (ta sama nazwa, a różne parametry). Często spotykaną praktyką jest możliwość przekazania wielu typów pod jedną zmienną.

In [ ]:
from typing import Union, Optional, Sequence, List

def func_opt(par1: Union[Sequence[str], str], par2: Optional[List[str]]=None):
    if isinstance(par1, list):
        print(f'list: {locals()}')
    elif isinstance(par1, str):
        print(f'str: {locals()}')
    else:
        print(f'other: {locals()}')

func_opt(['test', 'test2'])
func_opt('test')

Funkcja `func_opt` przyjmuje dwa parametry, wymagany `par1` oraz opcjonalny `par2`. Pierwszy z nich może przyjmować postać
dowolnego typu iteracyjnego np. lista, tupla, zbiór, oznaczonego w anotacji jako `sequence`. Druga możliwość to przekazanie do zmiennej `par1` łańcucha znaków. Użycie funkcji wbudowanej `locals` wyświetla wszystkie zmienne lokalne zadeklarowane w funkcji.

## 2. Dekoratory funkcji

Dekoratorem funkcji nazywamy funkcję przypisaną do deklaracji innej funkcji bazowej. W momencie wywołania funkcji bazowej w kodzie programu, która zawiera dekorator, sterowanie wywoła dekorator, którego zadaniem jest wykonanie dowolnych operacji i manualne wywołanie funkcji. Daje to ogromną możliwość wpływu na działanie samej 'udekorowanej' funkcji. Można dodawać parametry/modyfikować parametry czy przechwytywać wyjątki. Dość istotna jest tutaj niezależność obu funkcji (_losing coupling_). W tym konkretnym przykładzie chodzi przede wszystkim o to, żeby dekorator nie był dedykowany dla jednej konkretnej funkcji. Poniżej znajduje się przykład użycia dekoratora, który przechwytuje wyjątek. Ma to uzasadnienie jedynie w przypadku, gdy dekorator jest w stanie zwrócić poprawną wartość.

In [ ]:
import functools

def call_safe(func):
    def wrapper(*args, **kwargs):
        print(f'calling {func.__name__}')
        try:
            return func(*args, **kwargs)
        except Exception as e:
            print(f'error found: {e}')

        return None

    return wrapper

@call_safe
def base_func(par1: str):
    print(locals())
    raise Exception('Exception test')

base_func('test')

Każda funkcja może posiadać jeden lub więcej dekoratorów. W powyższym przykładzie `*arg` odpowiada przekazywanym parametrom pozycyjnym (przykładem jest tutaj `par1`), a `**kwargs` ma wartość inną, niż `None` jedynie, gdy funkcja przyjmuje parametr typu `**kwargs` (słownik z listą parametrów). Następny przykład ilustruje jak modyfikować parametry przekazywane do funkcji.

In [ ]:
import functools

def call_safe(func):
    def wrapper(*args, **kwargs):
        print(f'calling {func.__name__}')
        try:
            args = (*args, 'par2') # dodanie parametru do listy
            return func(*args, **kwargs)
        except Exception as e:
            print(f'error found: {e}')

        return None

    return wrapper

@call_safe
def base_func(par1: str, par2: str, **kwargs):
    print(par1)
    print(par2)
    print(kwargs)
    raise Exception('Exception test')

print('test without par2, kwargs')
base_func('test')

print('test without par2 with kwargs')
base_func('test', **{'add_par': 'add_par_val'})

Kolejny bardziej zaawansowany przykład prezentuje dekorator sprawdzający typy w momencie przekazywania parametrów do funkcji bazowej. Moduł standardowy `inspect` pozwala pobierać typy przekazywanych parametrów. Przykład pochodzi z książki: "Mark Summerfield, Advanced Python 3 Programming Techniques, Addison-Wesley Professional, 2009".

In [ ]:
import inspect
import functools
from typing import List

def strictly_typed(function):
    annotations = function.__annotations__
    arg_spec = inspect.getfullargspec(function)

    assert "return" in annotations, "missing type for return value"
    for arg in arg_spec.args + arg_spec.kwonlyargs:
        assert arg in annotations, ("missing type for parameter '" +
                                    arg + "'")
    @functools.wraps(function)
    def wrapper(*args, **kwargs):
        for name, arg in (list(zip(arg_spec.args, args)) +
                          list(kwargs.items())):
            assert isinstance(arg, annotations[name]), (
                    "expected argument '{0}' of {1} got {2}".format(
                    name, annotations[name], type(arg)))
        result = function(*args, **kwargs)
        assert isinstance(result, annotations["return"]), (
                    "expected return of {0} got {1}".format(
                    annotations["return"], type(result)))
        return result
    return wrapper

@strictly_typed
def test_funct(par1: str, count: int) -> str:
    return par1 * count

result = test_funct('str', 5)
print(result)

test_funct('str', 1.0)


## 3. Funkcje lokalne (zagnieżdżone)

Funkcje grupują powtarzające się fragmenty kodu w jednym miejscu. Ta sama zasada może dotyczyć długich funkcji, które mogą posiadać funkcje lokalne, dostępne jedynie z funkcji głównej. Technika ta ma uzasadnienie, gdy realizowany blok kodu zagnieżdżonej funkcji jest bardzo powiązany z funkcją główną i wydzielenie tej funkcji zupełnie osobno nie ma uzasadnienia (ze względu na swoją specyfikę, funkcja będzie zawsze wywoływana przez dokładnie jedną funkcję). Poniższy kod przedstawia sposób definiowania takich funkcji lokalnych.

In [ ]:
def func_main(par1: str):
    def op1():
        print('op1 called')
        pass

    print(par1)

    op1()

func_main('par1')

Budowanie funkcji zagnieżdżonych ma tę zaletę, że każda zmienna lokalna funkcji głównej jest dostępna w funkcji zagnieżdżonej. W przypadku modyfikacji tej zmiennej, należy dodać słowo kluczowe `nonlocal`, które informuje kompilator, że zmienna nie jest nową zmienną funkcji zagnieżdżonej, a zmienną funkcji głównej.

In [ ]:
def func_main():
    local_1, local_2 = 'local 1', 'local 2'

    def op1():
        nonlocal local_1
        print(f'op_1: local_1')
        print(f'op_1: local_2')

        local_1 = 'local_1_mod'
        local_2 = 'local_2_mod'

    op1()

    print(f'func_main: {local_1}')
    print(f'func_main: {local_2}')

func_main()

Zmienna `local2` w funkcji zagnieżdżonej `op1` jest zmienną lokalną tej funkcji i jej wartość nie jest przekazywana dalej.

Innym sposobem wykorzystania funkcji zagnieżdżonych jest optymalizacja wywołań rekurencyjnych. Przykładem może być tutaj generowanie ciągu Fibinacciego.

In [ ]:
def fib():
    a, b = 0, 1
    def inner_fib():
        nonlocal a,b
        a, b = b, a + b

        return b

    return inner_fib

f = fib()
for _ in range(1, 20):
    print(f())

Zmienna `f` nie tylko przechowuje adres funkcji `inner_fib`, ale również bieżący kontekst funkcji `f` (w tym przypadku zmienne `a`, `b`). Powyższy przykład łatwo przełożyć na większość funkcji rekurencyjnych, bez konieczności przejmowania się tzw. _long tail_ czyli wielokrotnym odkładaniu się na stos kolejnych kontekstów funkcji rekurencyjnej, co przekłada się na czas działania takiej funkcji.

## Generatory

Załóżmy, że mamy funkcję, która zwraca listę wartości, a lista ta jest następnie przekazana do pętli `for`. Wykonane zostaną następujące operacje, wywołanie funkcji, przekazanie wyniku do zmiennej lokalnej funkcji głównej (wywołującej), a następnie iterowanie po kolejnych elementach listy. Jeśli algorytm wyszukuje konkretną wartość na liście, bez względu na to, który to element listy, w pamięci zostanie zadeklarowana pełna tablica. Taki algorytm jest efektywny jedynie w przypadku, gdy wygenerowanie każdego elementu listy ma małą złożoność obliczeniową. Przyjmijmy, że mam przypadek drugi, w którym przeszukujemy zawartość dużych plików. Wylistowanie zawartości wszystkich plików, aby następnie po nich wyszukiwać jest nieefektywne. Pomocne są wtedy generatory, które nie kończą działania w momencie przekazania wartości do funkcji wywołującej. Kolejny przykład ilustruje sposób deklaracji generatora.

In [ ]:
def fun_gen(count):
    for i in range(count):
        print(f'iter: {i} in fun_gen')
        if i % 2 == 0:
            yield i

for i in fun_gen(10):
    print(f'iter: {i} in main')
    print(i)

Funkcja `fun_gen` zwraca po jednym elemencie liczbę parzystą z przedziału od zera do `count`. Sterowanie programu przechodzi między funkcję `fun_gen`, a instrukcją `for` kodu wywołującego funkcję `fun_gen`. Wcześniejsze zakończenie działania generatora może nastąpić po wywołaniu wyjątku z funkcji `fun_gen`, w kodzie wywołującym po wyjściu z pętli za pomocą słowa kluczowego `break` lub w samym generatorze po wywołaniu `return`.

In [ ]:
def fun_gen(count):
    for i in range(count):
        print(f'iter: {i} in fun_gen')
        if i % 2 == 0:
            yield i
        elif i % 3 == 0 and i > 0:
            return

for i in fun_gen(10):
    print(f'iter: {i} in main')
    print(i)

    if i > 100:
        break

Do otrzymania kolejnych elementów można użyć funkcji wbudowanej `next`, co prezentuje kolejny przykład.

In [ ]:
def fun_gen(count):
    for i in range(count):
        if i % 2 == 0:
            yield i

print(next(fun_gen(10)))

Realizacja generatorów odbywa się poprzez tzw. iteratory. W momencie kompilacji funkcji ze słowem kluczowym `yield`, kompilator tworzy iterator dla tej funkcji. Tworzenie własnych iteratorów jest możliwe implementując funkcje `__next__` we własnej klasie. Zadaniem tej funkcji jest przechowywanie aktualnie zwróconej wartości, a jej kolejne wywołanie, zwrócenie kolejnych rezultatów, tzn. dla tej samej instancji iteratora funkcja `__next__` wywoływana jest wielokrotnie.

Powyższe przykłady przedstawiały skończone generatory dane parametrem `count`. Można jednak tworzyć nieskończone generatory, co jest bardzo przydatne w momencie, gdy nie jest znana liczba elementów jakie należy wygenerować. Przykładem może tu być generowanie liczby pierwszej większej niż zadana liczba.

## Zadania do wykonania

### Zadanie 1

Napisz dekorator, który autoryzuje użytkownika na podstawie podanego loginu i hasła. Do przechowywania danych należy wykorzystać klasę `shelve`. Do maskowania hasła można użyć następującej funkcji:

In [ ]:
import hashlib, binascii, hmac
from os import urandom
import shelve

def open_db(func):
    def wrapper(*args, **kwargs):
        try:
            with shelve.open("login_auth.db") as shelf:
                args = (*args, shelf) if args else (shelf,)
                return func(*args, **kwargs)
        except Exception as e:
            print(f'Error found: {e}')
    return wrapper

@open_db
def salt_to_file(shelf):
    shelf['salt'] = hashlib.sha256(urandom(60)).hexdigest().encode('ascii')

@open_db
def add_user(login: str, password: str, shelf):
        shelf[login] = hash_password(password)

@open_db
def hash_password(password, shelf):
    """Hash a password for storing."""
    pwdhash = hashlib.pbkdf2_hmac('sha512', password.encode('utf-8'),
                                  shelf['salt'], 100000)
    pwdhash = binascii.hexlify(pwdhash)
    return (shelf['salt'] + pwdhash).decode('ascii')

@open_db
def auth_user(user: str, passwd: str, shelf):
    for id, key in shelf.items():
        if (str(id) == user) and (hmac.compare_digest(key, hash_password(passwd))):
             return f'User {user} is logged'

salt_to_file()

add_user('akLeszek', 'test1234')
add_user('zbyszek', 'zaq1234')
add_user('andrzej', 'qaz1234')
add_user('joda', 'qwertyuiop0987')

print(auth_user('akLeszek', 'test1234'))
print(auth_user('zbyszek', 'zaq1234'))
print(auth_user('andrzej', 'qaz1234'))
print(auth_user('joda', 'qwertyuiop0987'))

Należy pamiętać, że `salt` należy utworzyć raz i zachować w chronionym pliku.

### Zadanie 2

Napisz dekorator, który ogranicza argument funkcji do zadanego przedziału.

In [ ]:
def restrain(a: int, b: int):
   def wrapper(fun):
       def inner_wraper(*args):
           aa = a
           bb = b
           if args[0] > a:
               aa = args[0]
           if args[1] < b:
               bb = args[1]
           return fun(aa, bb)
       return inner_wraper
   return wrapper

@restrain(-2, 2)
def tmp(a, b):
    return a * b

print(tmp(-1, 3))

### Zadanie 3

Napisz generator, który dla zadanego wielomianu i przedziału wartości, zwróci jego miejsca zerowe. Dla pierwszego wywołania wynikiem ma być pierwsze miejsce zerowe, dla drugiego, drugie itd. Można zastosować naiwny algorytm sprawdzający kolejne wartości wielomianu. Można również zastosować kodowanie wielomianu bezpośrednio w kodzie języka Python: `x**n*x**(n-1)`.

In [ ]:
import numpy as np

def poli_calc(poly, range):
    polynomial = np.poly1d(poly)
    print(f'Zadany wielomian:\n{polynomial}\n')
    roots = polynomial.r
    # print(roots)
    for i in roots:
        if range[0] <= i <= range[1]:
            yield i

gen = poli_calc([1, 9, -2, -18, 0], [-1.5, 1.5])

print(next(gen))
print(next(gen))
print(next(gen))